In [1]:
import copy
import re
import os
import copy
import pandas as pd
import unidecode
import gensim
from stop_words import get_stop_words
from gensim.models import TfidfModel
from gensim.utils import simple_preprocess
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Store data
data_list = os.listdir('./data')
sum_list = []
for i in data_list:
    a = []
    dir = './data'+'/'+i
    with open(dir,'r',encoding = 'utf-8') as f:
        content = f.read()
    f.close()
    a.append(i.split('.')[0])
    a.append(content)
    sum_list.append(a)

df = pd.DataFrame(sum_list,columns = ['year','values'])
df = df.set_index('year')
value_data = df['values'].values.tolist()

In [3]:
# Clean data
remove_pattern = [
    (r'\s+[a-zA-Z]\s+', ' '),  # Remove all single characters
    (r'\^[a-zA-Z]\s+', ' '),
    (r'\d+', ''),  # Remove all numbers
    (r'\s+', ' '),  # Substitute multiple white spaces with single space
    (r'^b\s+', ''),  # Remove prefixed 'b'
                  ]

In [4]:
word_data = []

for abstract in range(0, len(value_data)):
    temp = unidecode.unidecode(re.sub(r'\W', ' ', str(value_data[abstract])))  # Remove all the special characters
    for p, r in remove_pattern:
        temp = re.sub(p, r, temp)
    word_data.append(temp)


def sentence_to_words(abstract):
    # If deacc=True then remove punctuations
    return (gensim.utils.simple_preprocess(str(abstract), deacc=True))

data_words = [sentence_to_words(x) for x in word_data]

In [7]:
# Get stop words list
stop_words = get_stop_words('english')

stop_words.extend(['actually', 'afterwards', 'almost', 'already', 'also', 'although', 'among', 'amongst', 'another', 'apart', 'around', 'aside', 'author', 'authors', 'away',
                   'become', 'becoming', 'better', 'beyond', 'certain', 'could', 'commonly', 'considerable', 'consider', 'de', 'definitely', 'eg', 'either', 'etc',
                   'f', 'g', 'h', 'hence', 'hereafter', 'herein', 'however', 'indeed', 'instead', 'illustrate', 'illustrates', 'demonstrate', 'demonstrates',
                   'important',  'j', 'k', 'likely', 'many', 'may', 'maybe', 'meanwhile', 'might', 'moreover', 'much', 'n', 'nevertheless', 'neither', 'normally', 'often',
                   'otherwise', 'particular', 'q', 'quite', 'understand', 'understood',
                   'rather', 'regardless', 'relatively', 'respectively', 'reveal', 'since', 'suggest', 'suggests', 'specifically', 'particularly', 'therefore', 'therein',
                   'though', 'thus', 'together', 'toward', 'towards', 'unless', 'upon', 'using', 'well', 'whereas', 'whereafter',
                   'whether',  'amico', 'edt', 'amp','mccarty_at_kcl', 'est', 'know', 'willard',
                   'htm',  'com', 'new', 'like', 'fqs',  'use', 'text', 'cch', 'bitnet', 'mccarty', 'kcl', 'html',
                    'org', 'ac', 'uk', 'edu', 'num', 'vol', 'date', 'href', 'www', 'http', 'will', 'ninch'])

def remove_stopwords(words):
    result = [x for x in words if x not in stop_words]
    return result

data_words_nostops = [remove_stopwords(x) for x in data_words] # Remove stop words

test = [" ".join(x) for x in data_words_nostops]

sum_list = copy.deepcopy(test)

In [8]:
# Instantiate tf instance
model = TfidfVectorizer(use_idf=True, smooth_idf=True, norm=None)
# Input training set matrix, each row represents one text
model_fit = model.fit_transform(sum_list)
word=model.get_feature_names()
tfidf_matrix=model_fit.toarray()


max_idf = tfidf_matrix.max(axis=0).tolist()
s_d = [(word[i], max_idf[i]) for i in range(len(max_idf))]
sortedList2 = sorted(s_d, key=lambda x:x[1])
sd = sortedList2[-10:]
print(sd)

for id_text in range(0, tfidf_matrix.shape[0]):
    rank = [i for i, v in sorted(list(enumerate(tfidf_matrix[id_text])), key=lambda x:x[1])]
    s=[(word[kl], tfidf_matrix[id_text][kl]) for kl in rank[-5:]]
    print(s)

[('utorepas', 1329.120536739557), ('research', 1399.0), ('computer', 1969.0), ('information', 2006.0), ('can', 2288.0), ('humanities', 2299.0), ('one', 2404.0), ('university', 2443.0), ('subject', 3604.0), ('humanist', 4495.0)]
[('subject', 1206.0), ('humanist', 1288.0), ('humanities', 1288.0), ('research', 1399.0), ('university', 1400.0)]
[('subject', 1071.0), ('information', 1264.0), ('university', 1294.0), ('humanities', 1550.0), ('humanist', 2757.0)]
[('humanities', 1132.0), ('information', 1143.0), ('subject', 1238.0), ('university', 1395.0), ('humanist', 1405.0)]
[('can', 659.0), ('software', 660.0), ('university', 707.0), ('subject', 920.0), ('humanist', 1418.0)]
[('one', 1209.0), ('can', 1322.0), ('university', 1892.0), ('subject', 2297.0), ('humanist', 2539.0)]
[('university', 2178.0), ('can', 2288.0), ('one', 2404.0), ('subject', 3604.0), ('humanist', 4443.0)]
[('wmccarty', 864.8123175954696), ('research', 970.0), ('digital', 1048.0), ('university', 1071.0), ('humanities', 11